# Recursion

So far, we have solved problems requiring an unbound number of steps (drawing a line of length $N$, where $N$ is given by input and thus unknown) with loops. Loops express very well "linear" processes, in which we accumulate the work done so far in the state, and at every iteration of the loop we simply add yet another "step" to the solution.

We will now focus on a comparable technique which, instead of using loops, makes use of functions in order to achieve the same result. Let us go back to a simple sample, a line of asterisks of length `n`, in order to have a problem to solve but without the excessive complexity of the problem getting in the way of our exploration.

Let us, once again, observe a line of six asterisks: `"******"`. Instead of splitting the line in a series of elementary elements, the steps, which are then assembled left-to-right (the individual asterisks), let us observe something else. A line can be built by concatenating other lines together! For example, the line above could be decomposed as:

- `"*" + "*****"`
- `"**" + "****"`
- `"***" + "***"`
- `"****" + "**"`
- `"*****" + "*"`

Let us focus on the very first of these decompositions: a line of length $n$ is made up of an asterisk, followed by another line of length $n-1$:

$$\underbrace{\texttt{"*...*"}}_{n} = \texttt{"*"} + \underbrace{\texttt{"*...*"}}_{n-1}$$

Of course, it does not really make much sense to define a line of negative length, which would be attempted by the definition above when $n=0$. In such cases, we skip the decomposition and we can simply produce an empty line, which happens to be the very same as an empty string:

$\underbrace{\texttt{"*...*"}}_{0} = \texttt{""}$$


Suppose we were defining `line` as a function, with a single parameter `n`, then we could attempt writing the above formula in code, as follows:

```
line = n =>
  if n = 0 then
    return ""
  else
    rest = line(n-1)
    return "*" + rest
```

The code above looks odd, but not unreasonable. Even more surprisingly, translation into Python produces an actually working program!

In [1]:
def line(n):
    if n == 0:
        return ""
    else:
        rest = line(n-1)
        return "*" + rest
print(line(int(input())))

6
******


Let us now briefly study the state trace of this program:
[[STATE TRACE]]

The stack grows, creating one entry for each asterisk to be added. When one of the intermediate calls to `line` returns, then we add its value to the return expression, which grows "right-to-left" in a cascade of further `return` instructions which assemble the final line.

We have written our first _recursive_ function.

## Aggressive splitting
Of all the possible decompositions we have seen above, the one in the middle (`"***" + "***"`) is perhaps even more interesting than the one we have just implemented. The main attraction of this decomposition is its simmetry, that is a line of length $N$ can be thought of as the combination of two lines of length $N/2$.

Therefore, we can restate this new piece of knowledge about lines:

$$\underbrace{\texttt{"*...*"}}_{n} = \underbrace{\texttt{"*...*"}}_{n/2} + \underbrace{\texttt{"*...*"}}_{n/2}$$

Unfortunately, there is no real way to split an asterisk in two, so the above is understood as true only if $n$ is even. If $n$ were odd, then we would need an extra asterisk in the middle:

$$\underbrace{\texttt{"*...*"}}_{n} = \underbrace{\texttt{"*...*"}}_{n/2} + \texttt{"*"} \underbrace{\texttt{"*...*"}}_{n/2}$$

The last, minor addition to our brief dissertation about the nature of lines is that for $n=0$ we get an empty line, which does not need to be decomposed into other lines:

$\underbrace{\texttt{"*...*"}}_{0} = \texttt{""}$$

We could recap this formulation in a tighter format by writing as follows:

\begin{align*}
\underbrace{\texttt{"*...*"}}_{n} &= \texttt{""} & \text{ when } n=0 \\
&= \underbrace{\texttt{"*...*"}}_{n/2} + \underbrace{\texttt{"*...*"}}_{n/2} & \text{ when } n>0 \wedge n \text{ is even}  \\
&= \underbrace{\texttt{"*...*"}}_{n/2} + \texttt{"*"} + \underbrace{\texttt{"*...*"}}_{n/2} & \text{ when } n>0 \wedge n \text{ is odd}  \\
\end{align*}

This decomposition turns gets translated into a function quite directly: the checks to left-hand-side of the equations above turn into one or more conditional statements. The smaller lines we decompose into have become a call from `line` to `line` itself. The resulting code is:

```
line = n =>
  if n = 0 then
    return ""
  else
    half = line(n//2)
    extra = if n % 2 = 0 then "" else "*"
    return half + half + extra
```

Interestingly enough, this way of reasoning produces a significant improvement in our implementation of lines: now at every step we reduce the problem by a factor of two, so there are far less steps involved than in the previous implementations that added one single asterisk at a time!

In [4]:
def line(n):
    if n == 0:
        return ""
    else:
        half = line(n//2)
        extra = "" if n % 2 == 0 else "*" 
        return half + half + extra
print(line(int(input())))

7
*******


- let us now see the state trace
    - [[STATE TRACE]]
    - the stack grows, creating one entry per active function
        - since we divide the length by two every time, it takes a lot less total steps
    - when a function returns, then we add its value to the return expression, which doubles at each level

## General idea of the process
- the general process we are following is called "recursion", or (in mathematics) "induction"
    - state the problem: _drawing a line of `n` characters_
    - divide the problem into similar, smaller problems (the _inductive_ case)
        - a line is made up of smaller lines
    - identify an end case that is easy to identify and solve (the _base_ case)
        - a line of $0$ characters is the empty string `""`
    - define a function to solve the problem: `line = n => ...`
    - check the input of the function to see if we are dealing with the end case or not:
        - `if n = 0 then "" else ...`
    - in the `else` branch, invoke the function recursively with adjusted parameters (**closer to the base case**):
        - `rest = line(n-1)`
    - assemble the partial results in the final result and return it
        - `return "*" + rest`
- following our template, a recursive function typically looks as follows:


```
f = args =>
  if simple_to_solve then
    return base_solution
  else
    part1 := f(args1)
    part2 := f(args2)
    ...
    partN := f(argsN)
    return combine_parts
```

- in the case of drawing lines:
    - `f` becomes `line`
    - `args` becomes `n`
    - `simple_to_solve` becomes `n = 0` (or `n <= 0` to also gracefully capture nonsensical calls such as `line(-3)`)
    - `base_solution` becomes `""`
    - `part1` becomes `"*"`
    - `part2` becomes `rest`
    - `f(args2)` becomes `line(n-1)`
    - `combine_parts` becomes `part1 + part2`

## Power of recursion
- the question now becomes: is recursion more or less (or equally) powerful as loops?
- to answer this, let us begin with a simple loop that sums the numbers from $1$ to $n$

```
n := int(input())
sum := 0
while n > 0 do
  sum := sum + n
  n := n - 1
print(sum)
```

- we could also rephrase the problem in recursive terms
    - the sum of all numbers $1 \dots n$ is:
        - $0$ if $n = 0$
        - the sum of all numbers $1 \dots n-1$ plus $n$ otherwise
- this leads us to the following implementation:

```
sum_numbers_to = n =>
  if n <= 0 then
    return 0
  else
    rest := sum_numbers_to(n-1)
    return rest + n
    
sum := sum_numbers_to(int(input())
print(sum)
```

- at a first glance, the two implementations look different, yet comparable
- let us see if we can bridge the gap between them
- the first line before the `while` loop, `sum := 0`, is the base case (`BASE`)
- the condition `n > 0` identifies when the base case _is not already met_ and we must proceed with further computation (`COND`)
- the body of the loop contains the argument to the recursive call `n-1` (`REC`)
- the body of the loop also contains the composition of the final result (`COMP`)

- we can now restate both versions of the program, but the elements of code for their names

- the loop becomes now:
```
n := int(input())
sum := BASE
while COND do
  sum := COMP
  n := REC
``` 

- the recursive version becomes:
```
sum_numbers_to = n =>
  if not COND then
    return BASE
  else
    rest := sum_numbers_to(REC)
    return COMP
```

- since virtually all individual loops can be condensed to a format such as the above (minus some minor adjustments), then we know that `while`, as a language construct, can be fully emulated with a recursive function
- we can now check whether or not the opposite is the case as well

- we can convert a recursive function with the same structure as the one above to a loop, but there are other recursive functions that are not really easy to translate
- let us gor example consider the Fibonacci function, which models the growth of a population of rabbits:

```
fib = n => if n <= 1 then return n else return fib(n-1) + fib(n-2)
```

- [[STATE TRACE]]
- the stack is used very intensively, because every level adds two fully recursive calls
- the program also grows a lot in size, because of all the intermediate steps that are frozen in place
    - because of its huge size, this formulation cannot be directly translated into a loop (unless we add a global variable storing a stack, but then we are simply rebuilding recursive functions!)
    - a loop "grows" linearly in complexity, a recursive function exponentially!
- the power of recursion makes it a fundamental tool in the toolbox of a modern developer!